# Prep

In [1]:
import pandas as pd

In [2]:
GOLD_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b3-gold'
VIEWS_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b4-views'

# Functions

## Transform

# Extract

In [3]:
df_workouts = pd.read_csv(f"{GOLD_FOLDER}/workouts.csv", sep=";", decimal=',')
df_exercises_log = pd.read_csv(f"{GOLD_FOLDER}/exercises_log.csv", sep=";", decimal=',')
df_exercises_ref = pd.read_csv(f"{GOLD_FOLDER}/exercises_ref.csv")

# Transform

## Single Sets

In [4]:
df_single_sets = (
    df_exercises_log
    .merge(
        df_exercises_ref[['exercise_name', 'muscle_group', 'exercise_group', 'main_muscle']],
        how='left',
        on=('exercise_name')
        )
    .merge(
        df_workouts[['id', 'start_time']],
        how='left',
        left_on=('workout_id'),
        right_on=('id')
    )
    .drop(columns=['id'])
    .astype({'start_time': 'datetime64[ms]'})
)

In [5]:
df_single_sets['week'] = df_single_sets['start_time'].apply(lambda x: x - pd.Timedelta(days=(x.weekday())))
df_single_sets['month'] = df_single_sets['start_time'].dt.to_period('M').dt.start_time

In [6]:
df_single_sets = df_single_sets[[
    'workout_id',
    'start_time',
    'week',
    'month',
    'exercise_number',
    'exercise_name',
    'equipment',
    'muscle_group',
    'exercise_group',
    'main_muscle',
    'goal',
    'measure',
    'set_number',
    'weight',
    'total_weight',
    'quantity',
    'volume'
]]

In [7]:
#df_single_sets.head()

## Complete Sets

In [8]:
df_complete_sets = (
   df_single_sets
   .groupby([
      'workout_id',
      'start_time',
      'week',
      'month',
      'exercise_number', 
      'exercise_name', 
      'equipment', 
      'muscle_group', 
      'exercise_group', 
      'main_muscle', 
      'goal', 
      'measure'])
   .agg(
      sets = ('set_number', 'max'),
      min_weight = ('total_weight', 'min'),
      max_weight = ('total_weight', 'max'),
      avg_weight = ('total_weight', lambda x: round(x.mean(), 1)),
      stdev_weight = ('total_weight', lambda x: round(x.std(), 1)),
      min_quantity = ('quantity', 'min'),
      max_quantity = ('quantity', 'max'),
      avg_quantity = ('quantity', lambda x: round(x.mean(), 1)),
      stdev_quantity = ('quantity', lambda x: round(x.std(), 1)),
      total_quantity = ('quantity', 'sum'),
      total_volume = ('volume', 'sum'))
   .reset_index()
)

In [9]:
df_complete_sets

,workout_id,start_time,week,month,exercise_number,exercise_name,equipment,muscle_group,exercise_group,main_muscle,...,min_weight,max_weight,avg_weight,stdev_weight,min_quantity,max_quantity,avg_quantity,stdev_quantity,total_quantity,total_volume
0,20231014949,2023-10-14 09:49:00,2023-10-09 09:49:00,2023-10-01,1,Leg Press,Machine,Lower Body,Leg Press,Quads,...,136.0,140.0,138.7,2.3,12,15,13.0,1.7,39,5400.0
1,20231014949,2023-10-14 09:49:00,2023-10-09 09:49:00,2023-10-01,2,Standing Calf Raises,Machine,Lower Body,Calf Raise / Press,Calves,...,130.0,140.0,135.3,4.5,15,15,15.0,0.0,90,12180.0
2,20231014949,2023-10-14 09:49:00,2023-10-09 09:49:00,2023-10-01,3,Hip Adduction,Machine,Lower Body,Hip Abduction,Abductors,...,50.0,55.0,53.3,2.6,6,15,10.0,4.1,60,3150.0
3,20231014949,2023-10-14 09:49:00,2023-10-09 09:49:00,2023-10-01,4,Hip Abduction,Machine,Lower Body,Hip Abduction,Abductors,...,35.0,40.0,38.3,2.6,10,15,12.3,2.3,74,2810.0
4,20231014949,2023-10-14 09:49:00,2023-10-09 09:49:00,2023-10-01,5,Leg Extensions,Machine,Lower Body,Leg Extension,Quads,...,50.0,55.0,53.3,2.9,12,15,13.0,1.7,39,2070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,202501061420,2025-01-06 14:20:00,2025-01-06 14:20:00,2025-01-01,8,Curls,Cable,Upper Body,Biceps Curl,Biceps,...,25.0,25.0,25.0,0.0,12,15,14.0,1.5,168,4200.0
1055,202501071229,2025-01-07 12:29:00,2025-01-06 12:29:00,2025-01-01,2,Romanian Deadlifts,Barbell,Lower Body,Deadlift,Glutes,...,10.0,10.0,10.0,0.0,15,15,15.0,0.0,45,450.0
1056,202501071229,2025-01-07 12:29:00,2025-01-06 12:29:00,2025-01-01,3,Standing Calf Raises,Bodyweight,Lower Body,Calf Raise / Press,Calves,...,5.0,5.0,5.0,0.0,15,15,15.0,0.0,90,450.0
1057,202501071229,2025-01-07 12:29:00,2025-01-06 12:29:00,2025-01-01,4,Leg Extensions,Machine,Lower Body,Leg Extension,Quads,...,55.0,60.0,58.3,2.9,15,15,15.0,0.0,45,2625.0


## Workouts

# Save

In [10]:
df_single_sets.to_csv(f"{VIEWS_FOLDER}/single_sets.csv", sep=";", decimal=',', index=False)
df_complete_sets.to_csv(f"{VIEWS_FOLDER}/complete_sets.csv", sep=";", decimal=',', index=False)